In [ ]:
from src.guess.guess import ConfigType, Guess, Config
from src.domain.datasets.UnixCommandDataset import UnixCommandDataset  # Register dataset
from src.domain.datasets.ProjectEulerDataset import ProjectEulerDataset
from src.domain.datasets.BringUpDataset import BringUpDataset

prediction = None

config = Config(ConfigType.QWEN_X862ARM64.get_path())
guess = Guess(config=config)

test_instance = guess.data_loader.iter().__next__()

result = guess.model.predict(test_instance, guess.inference_cfg)

2025-05-10 23:04:36,535 - src.domain.models.QwenModel - INFO - Initializing QwenModel with ahmedheakl/ex19_qwen2.5-1.5b-1m-stack-16kcw on device mps


Loading Qwen model: ahmedheakl/ex19_qwen2.5-1.5b-1m-stack-16kcw


2025-05-10 23:04:43,112 - QwenModel - INFO - Initialized QwenModel on device: mps:0
2025-05-10 23:04:43,113 - src.domain.models.QwenModel - INFO - Model initialization completed in 6.58 seconds


Source JSONL: data/processed/X86/BringUp_x86.jsonl
Target JSONL: data/processed/ARM64/BringUp_arm.jsonl
Loading file: data/processed/X86/BringUp_x86.jsonl
Loaded 31 entries from data/processed/X86/BringUp_x86.jsonl
Loading file: data/processed/ARM64/BringUp_arm.jsonl
Loaded 31 entries from data/processed/ARM64/BringUp_arm.jsonl


AttributeError: 'generator' object has no attribute 'next'

In [ ]:
from src.helpers.torch_unpicker import CPU_Unpickler

predictions = CPU_Unpickler(open("bringup_wAttention_Tokenized.pkl", "rb")).load()
print(predictions.keys())

dict_keys(['fy-shuffle.s', 'bloom-filter.s', 'strange.s', 'topo-sort.s', 'weekday.s', 'knights-tour.s', 'murmur-hash.s', 'spirograph.s', 'gcd-list.s', 'skeleton.s', 'simple-grep.s', 'pi-calc.s', 'indirect-test.s', 'flood-fill.s', 'priority-queue.s', 'bubble-sort.s', 'ackermann.s', 'spelt2num.s', 'totient.s', 'cipher.s', 'mersenne.s', 'heapsort.s', 'sieve.s', 'hanoi.s', 'natlog.s', 'rle-compress.s', 'quine.s', 'mandelbrot.s'])


In [6]:
from src.sketch.sketch import Sketch

sketch = Sketch(config, guess.model)

In [54]:
from typing import List
from src.sketch.sections import Section
from src.sketch.sketch import Sketch
from src.helpers.model import PredictionResult
import numpy as np
import torch


def section_confidence(section: Section, prediction: PredictionResult) -> float:
    return prediction.confidence[section.start:section.end].mean()


def remove_sections(sections: List[Section], prediction: PredictionResult) -> PredictionResult:
    pred = prediction.pred[0]
    confidence = prediction.confidence
    alignments = prediction.alignments

    print(len(pred))
    print(len(confidence))
    print(len(alignments))

    sections.sort(key=lambda section: section.start)
    
    mask = np.ones(len(pred), dtype=bool)
    
    for section in sections:
        mask[section.start:section.end] = False
    
    filtered_pred = pred[mask]
    filtered_confidence = confidence[mask]
    filtered_alignments = alignments[mask] if alignments is not None else None
    
    return PredictionResult(
        instance_id=prediction.instance_id,
        source=prediction.source,
        pred=[filtered_pred],
        alignments=filtered_alignments,
        confidence=filtered_confidence
    )


def fix_duplicate_sections(sketch: Sketch, prediction: PredictionResult) -> PredictionResult:
    cleaned_pred = prediction

    sections = sketch.extract_sections(prediction.pred)
    sections_by_name = {}
    for section in sections:
        if section.name in sections_by_name:
            sections_by_name[section.name].append(section)
        else:
            sections_by_name[section.name] = [section]
    
    for section_name, sections in sections_by_name.items():
        if len(sections) > 1:
            sections.sort(key=lambda section: section_confidence(section, prediction), reverse=True)

            cleaned_pred = remove_sections(sections[1:], cleaned_pred)

    return cleaned_pred


In [56]:
from src.helpers.model import PredictionResult

for instance_id, prediction in predictions.items():
    print(instance_id)

    print(len(prediction.pred))
    print(len(prediction.alignments))
    print(len(prediction.confidence))

    pred = PredictionResult(
        instance_id=instance_id,
        source=[torch.tensor(prediction.source, dtype=torch.long)],
        pred=[torch.tensor(prediction.pred, dtype=torch.long)],
        alignments=torch.tensor(prediction.alignments, dtype=torch.long),
        confidence=torch.tensor(prediction.confidence, dtype=torch.float32)
    )
    
    fixed_pred = fix_duplicate_sections(sketch, pred)


fy-shuffle.s
3705
100
7410
bloom-filter.s
4019
100
8038
strange.s
978
100
1956
topo-sort.s
4743
100
9486
weekday.s
3082
100
6164
knights-tour.s
4814
100
9628
4814
9628
100


IndexError: The shape of the mask [4814] at index 0 does not match the shape of the indexed tensor [9628] at index 0